In [1]:
import warnings
warnings.filterwarnings(action='ignore')
%config Computer.use_jedi = False
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import missingno as msno
plt.rc('font', family='D2Coding')
from plotnine import *

In [17]:
academy = pd.read_csv('./data/동북부학원수.csv')
academy = academy.drop(columns=['Unnamed: 0'], axis=1)
academy

,시군구,법정동,면적(㎢)당 학원 수
0,강북구,미아동,15.331808
1,강북구,번동,4.059041
2,강북구,수유동,5.929487
3,강북구,우이동,0.194553
4,노원구,공릉동,3.037667
5,노원구,상계동,6.377551
6,노원구,월계동,4.215457
7,노원구,중계동,37.307692
8,노원구,하계동,8.252427
9,도봉구,도봉동,0.523560


In [11]:
width = pd.read_csv('./data/동북부면적.csv')
width = width.drop(columns=['Unnamed: 0'], axis=1)
width

,시군구,법정동,면적
0,강북구,미아동,4370.0
1,강북구,번동,2710.0
2,강북구,수유동,6240.0
3,강북구,우이동,10280.0
4,노원구,공릉동,8230.0
5,노원구,상계동,15680.0
6,노원구,월계동,4270.0
7,노원구,중계동,5200.0
8,노원구,하계동,2060.0
9,도봉구,도봉동,9550.0


In [12]:
house = pd.read_csv('./data/동북부집값.csv')
house = house.drop(columns=['Unnamed: 0'], axis=1)
house

,시군구,법정동,면적(㎡)당금액
0,강북구,미아동,990.0
1,강북구,번동,922.0
2,강북구,수유동,965.0
3,강북구,우이동,755.0
4,노원구,공릉동,1016.0
5,노원구,상계동,1115.0
6,노원구,월계동,1213.0
7,노원구,중계동,1169.0
8,노원구,하계동,1222.0
9,도봉구,도봉동,784.0


In [19]:
school = pd.read_csv('./data/동북부학교수.csv')
school = school.drop(columns=['Unnamed: 0'], axis=1)
school

,시군구,법정동,학교명
0,강북구,미아동,6
1,강북구,수유동,1
2,노원구,공릉동,2
3,노원구,상계동,6
4,노원구,월계동,5
5,노원구,중계동,9
6,노원구,하계동,3
7,도봉구,도봉동,2
8,도봉구,방학동,1
9,도봉구,쌍문동,4
